In [ ]:
!git clone https://github.com/Azzam-Radman/Toxic-Spans-Detection.git

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)

In [2]:
df1 = pd.read_excel('/content/Toxic-Spans-Detection/src/dataset/tokenized_1.xlsx')
df2 = pd.read_excel('/content/Toxic-Spans-Detection/src/dataset/tokenized_ready.xlsx')

In [3]:
df1 = df1.iloc[:1798, :].reset_index(drop=True)
df2 = df2.iloc[1798:, :].reset_index(drop=True)
df = pd.concat([df1, df2], axis=0).reset_index(drop=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,133,134,135,136,137,138,139,140,141,142
0,اربد,فيها,جامعات,اكثر,من,عمان,...,وفيها,قد,عمان,...,pad,pad,pad,pad,pad,pad,pad,pad,pad,pad
1,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,الحلو,انكم,بتحكوا,على,اساس,انو,الاردن,ما,فيه,فساد,...,pad,pad,pad,pad,pad,pad,pad,pad,pad,pad
3,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,كله,رائع,بجد,ربنا,يكرمك,pad,pad,pad,pad,pad,...,pad,pad,pad,pad,pad,pad,pad,pad,pad,pad


In [4]:
def remove_pad_nan(word_row, label_row):
    
    word_list = word_row.tolist()
    label_list = label_row.tolist()
    
    word_list_cleaned = [word for word in word_list if word != 'pad']
    label_list_cleaned = [label for label in label_list if label is not np.nan]
    
    pair = [word_list_cleaned, label_list_cleaned]
    return pair

In [5]:
pairs = []
for i in range(len(df)):
    if i%2 == 0:
        pairs.append(remove_pad_nan(df.iloc[i, :], df.iloc[i+1, :]))

In [6]:
counter = 0
for pair in pairs:
    if len(pair[0]) != len(pair[1]):
        counter += 1
        print(pair)
print(counter)

0


In [7]:
pairs_with_spaces = []

for pair in pairs:
    words_with_spaces = []
    labels_with_spaces = []
    len_one_pair = len(pair[0])
    
    for i in range(len_one_pair):
        words_with_spaces.extend([pair[0][i], ' '])
        labels_with_spaces.extend([pair[1][i], ' '])
        
    new_pair = [words_with_spaces, labels_with_spaces]
    pairs_with_spaces.append(new_pair)

In [8]:
counter = 0
for pair in pairs_with_spaces:
    if len(pair[0]) != len(pair[1]):
        counter += 1
        
print(counter)

0


In [9]:
# remove last space
for pair in pairs_with_spaces:
    pair[0].pop(-1)
    pair[1].pop(-1)

In [10]:
counter = 0
for pair in pairs_with_spaces:
    if len(pair[0]) != len(pair[1]):
        counter += 1
        
print(counter)

0


In [11]:
for pair in pairs_with_spaces:
    # length of the words or labels list
    len_one_pair = len(pair[0])
    # initialize the toxic spans list and a counter
    toxic_spans_list = []
    counter = 0
    # loop over each item in each list
    for idx in range(len_one_pair):
        # extract word label from the label list
        word_label = pair[1][idx]
        # get the word length from the word list
        if isinstance(pair[0][idx], str):
            len_word = len(pair[0][idx])
        else: 
            len_word = 1
            
        if word_label == 0:
            counter += len_word
        elif word_label == ' ':
            counter += 1
        else:
            toxic_spans = list(range(counter, counter+len_word))
            toxic_spans_list.extend(toxic_spans)
            counter += len_word
            
    pair.append(toxic_spans_list)

In [12]:
last_pairs = []
for pair in pairs_with_spaces:
    sentence = ''.join(map(str, pair[0]))
    spans = pair[-1]
    new_last_pair = [sentence, spans]
    last_pairs.append(new_last_pair)

In [13]:
zeros = np.zeros((len(df)//2, 2))
train_df = pd.DataFrame(zeros, columns=['Sentence', 'Spans'])

In [14]:
train_df['Spans'] = train_df['Spans'].astype('str')
for i in range(len(last_pairs)):
    train_df.iloc[i, 0] = last_pairs[i][0]
    train_df.iat[i, 1] = last_pairs[i][1]

In [15]:
train_df[1200: 1220]

,Sentence,Spans
1200,لا تحزن الدنيا ما فيها اشي تحزن عليه,[]
1201,لا تحزن ان الله معك,[]
1202,لا تحزن على دنا فانيه بل احزن على اخره باقيه,[]
1203,لا تخسر قيمتك بكلمه ، ولا تفقد احترامك بزله ، ولا تجعل همك في الدنيا هو حب الناس لك ، فالناس قلوبهم متقلبه قد تحبك اليوم و تكرهك غدا,[]
1204,لا تعليق,[]
1205,لا تفهمي غلط حبيبتي,[]
1206,لا تقنطوا من رحمه الله فهي وسعت كل شيء مهما عصينا فلنا رب كبير رحيم ورحمن,[]
1207,لا تياسوا من روح الله ، الله تعالى لا يعجزه شيء ، فكم من مريض شفاه ، وكم من فقير اغناه ، وكم من مشرف على الهلاك نجاه ، وكم من ضال هداه,[]
1208,لا حول ولا قوه الا بالله اذا اخذ ما اوهب اسقط ما اوجب,[]
1209,لا حول ولا قوه الا بالله العظيم لم اكن اعلم ان نسبه ذكاء العرب منخفضه جدا الى هذا الحد,"[52, 53, 54, 55, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 70, 71, 72, 74, 75, 76, 78, 79, 80, 82, 83, 84, 85]"


In [16]:
train_df.to_csv('train_ready.csv', index=False)